# Hints

<table style="width:90%">
    <tr><td style="text-align:left;"><a href="./06 - Complex Workflows.ipynb">Previous (Complex Workflows)</a><td style="text-align:right;"><a href="./08 - Solution1.ipynb">Next (Solution 1)</a></td></tr>
</table>

Here we present some hints and a solution to the previous exercise.

<details>
    <summary> Click to see the first hint</summary>
    The first step is to determine which functions should run locally and which ones should run in parallel on a more powerful machine. 
    <ul><li>From the table we can see that <code>split_image</code> and <code>make_catalog</code> are run only once and are good candidates for local execution.</li>
    <li>All of the other functions are run at least three times (once per color) and are good candidates for remote execution.</li>

</details>

<details>
    <summary> Click to see the second hint</summary>
    You will want to convert some of the functions to be Python Apps. The first targets will be those that will run locally:
    <ul><li> split_image</li>
        <li> make_catalog</li></ul>
<pre><code>
@python_app(executors=['local_tpex'])
def split_image() -> tuple:
    import skimage.io
    myimage = skimage.io.imread(images[0].filepath).astype(np.uint8)
    r = myimage[:,:,0]
    g = myimage[:,:,1]
    b = myimage[:,:,2]
    return r, g, b
</pre></code>
For this function we change the input from a string to a File object so that Parsl wraps it as well. This is strictly not necessary for local execution, but is good practice. Also not the local import as well.    
    
<pre><code>
@python_app(executors=['local_tpex'])
def make_catalog(tables, labels, outputs=[]):
    from astropy.io import fits
    hdus = [fits.PrimaryHDU()]
    counts = {}
    for i, v in enumerate(tables):
        counts[labels[i]] = v['Amplitude'].values.shape[0]
        col1 = fits.Column(name='Amp', format='E', array=v['Amplitude'].values)
        col2 = fits.Column(name='Xo', format='E', array=v['Xo'].values)
        col3 = fits.Column(name='Yo', format='E', array=v['Yo'].values)
        col4 = fits.Column(name='sigmaX', format='E', array=v['sigmaX'].values)
        col5 = fits.Column(name='sigmaY', format='E', array=v['sigmaY'].values)
        col6 = fits.Column(name='theta', format='E', array=v['theta'].values)
    
        hdu = fits.BinTableHDU.from_columns([col1, col2, col3, col4, col5, col6])
        hdu.header['band'] = labels[i]
        hdus.append(hdu)

    hdul = fits.HDUList(hdus)
    hdul.writeto(outputs[0].filepath, overwrite=True)
    
    print("Found these object counts")
    for i, v in counts.items():
        print(f"{i}: {v}")
</code></pre>
</details>

<details>
    <summary> Click to see the third hint</summary>
    
Now the rest of the functions should be converted to Python Apps as well. For any functions where the code does not change only the signature ans imports are being given. If you have access to an HPC system, these apps could be executed there).
<pre><code>
@python_app(executors=['local_tpex'])
def find_peaks(in_img, min_dist = 10):
    import numpy as np
    import math
    import skimage.feature as feature
    def intersect(c1: list, c2: list, radius: float) -> bool:
        d = math.sqrt((c1[0] - c2[0]) * (c1[0] - c2[0]) +
                      (c1[1] - c2[1]) * (c1[1] - c2[1]))

        if d < (2*radius * .9):
            return True
        return False
    ...
    
@python_app(executors=['local_tpex'])
def make_cutouts(in_img, peaks, radius=10):
    import numpy as np
    ...
   
@python_app(executors=['local_tpex'])
def make_fits(input_data):
    import pandas as pd
    import numpy as np
    import scipy.optimize as opt
    
    (images, centers, offsets) = input_data
    
    def twoD_Gaussian(point: tuple, amplitude: float, xo: float, yo: float, sigma_x: float, sigma_y: float, theta: float):
        (x, y) = point
        xo = float(xo)                                                              
        yo = float(yo)
        a = (np.cos(theta) ** 2) / (2 * sigma_x ** 2) + (np.sin(theta) ** 2) / (2 * sigma_y ** 2)   
        b = -(np.sin(2 * theta)) / (4 * sigma_x ** 2) + (np.sin(2 * theta)) / (4 * sigma_y ** 2)    
        c = (np.sin(theta) ** 2) / (2 * sigma_x ** 2) + (np.cos(theta) ** 2) / (2 * sigma_y ** 2)   
        gaussian = amplitude * np.exp(-(a * ((x - xo) ** 2) + 2 * b * (x - xo) * (y - yo) + c * ((y - yo) ** 2)))                                   
        return gaussian.ravel()
    ...

@python_app(executors=['local_tpex'])
def make_image(in_img, fit, label, outputs=[]):
    import numpy as np
    import matplotlib.pyplot as mpl
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    
    def twoD_Gaussian(point: tuple, amplitude: float, xo: float, yo: float, sigma_x: float, sigma_y: float, theta: float):
        (x, y) = point
        xo = float(xo)                                                              
        yo = float(yo)
        a = (np.cos(theta) ** 2) / (2 * sigma_x ** 2) + (np.sin(theta) ** 2) / (2 * sigma_y ** 2)   
        b = -(np.sin(2 * theta)) / (4 * sigma_x ** 2) + (np.sin(2 * theta)) / (4 * sigma_y ** 2)    
        c = (np.sin(theta) ** 2) / (2 * sigma_x ** 2) + (np.cos(theta) ** 2) / (2 * sigma_y ** 2)   
        gaussian = amplitude * np.exp(-(a * ((x - xo) ** 2) + 2 * b * (x - xo) * (y - yo) + c * ((y - yo) ** 2)))                                   
        return gaussian.ravel()
    ...
    
</code></pre>
</details>

<details>
    <summary> Click to see the third hint in an alternative form</summary>
Here we will create a single Python App that will encapsulate all the work to be done on a remote system.
<pre><code>
@python_app(executors=['local_tpex'])
def remote_run(in_img, label, min_dist, outputs=[]):
    import math
    import numpy as np
    import skimage.feature as feature
    import pandas as pd
    import scipy.optimize as opt
    import matplotlib.pyplot as mpl
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    def intersect(c1, c2, radius):
        ...
        
    def twoD_Gaussian(point: tuple, amplitude: float, xo: float, yo: float, sigma_x: float, sigma_y: float, theta: float):
        ...
        
    def find_peaks():
        ...
    
    def make_cutouts(peaks):
        ...
    
    def make_fits(input_data):
        (images, centers, offsets) = input_data
        ...
        
    def make_image(fit):
        ...
        
    img = in_img.astype(np.float32) / 255.
    pks = find_peaks()
    data = make_cutouts(pks)
    star_fits = make_fits(data)
    make_image(star_fits)
    return star_fits

</code></pre>
</details>

<details>
    <summary> Click to see the final hint</summary>
Here we need to write the code that launches the Parsl Apps.
<pre><code>

lbls = ["red", "green", "blue"]
params = []
image = File(os.path.join(os.getcwd(), "noao-m34.png"))

si = split_image(inputs=[image])

fts = []
made_images = []
for i, im in enumerate(si.result()):
    pks = find_peaks(im)
    mc = make_cutouts(im, pks.result())
    fts.append(make_fits(mc.result()))
    made_images.append(make_image(im, fts[i].result(), lbls[i], outputs=[File(os.path.join(os.getcwd(), lbls[i] + "_compare.png"))]))

for ft in fts:
    params.append(ft.result())

for mi in made_images:
    mi.result()
    
mkcat = make_catalog(params, lbls, outputs=[File(os.path.join(os.getcwd(), 'parameters.fits'))])
mkcat.result()

</code></pre>
</details>

<details>
    <summary> Click to see an alternative form of the final hint</summary>
Here we need to write the code that launches the Parsl Apps.
    <pre><code>
lbls = ["red", "green", "blue"]
params = []
image = File(os.path.join(os.getcwd(), "noao-m34.png"))

si = split_image(inputs=[image])
jobs = []
for i, im in enumerate(si.result()):
    jobs.append(remote_run(im, lbls[i], 10, outputs=[File(os.path.join(os.getcwd(), lbls[i] + "_compare.png"))]))

for job in jobs:
    params.append(job.result())
mkcat = make_catalog(params, lbls, outputs=[File(os.path.join(os.getcwd(), 'parameters.fits'))])
mkcat.result()

</code></pre>
</details>

Click <a href="08 - Solution1.ipynb">here</a> for a full solution.<BR>
Click <a href="09 - Solution2.ipynb">here</a> for an alternative form.
    
<table style="width:90%">
    <tr><td style="text-align:left;"><a href="./06 - Complex Workflows.ipynb">Previous (Complex Workflows)</a><td style="text-align:right;"><a href="./08 - Solution1.ipynb">Next (Solution 1)</a></td></tr>
</table>
